In [10]:
import pandas as pd
import numpy as np
import time
pd.set_option("max_columns", None)

df = pd.read_csv('kddcup99.csv')
df.drop(df[ (df.label != 'normal') & (df.label != 'smurf') ].index, inplace=True)
df = df.iloc[:5000,:].append(df[df['label'] == 'smurf'].iloc[:5000,:], ignore_index=True)

In [11]:
df = pd.concat([pd.get_dummies(df['protocol_type'], prefix='protocol_type'), df], axis=1)
df.drop('protocol_type', axis=1, inplace=True)

df = pd.concat([pd.get_dummies(df['service'], prefix='service'), df], axis=1)
df.drop('service', axis=1, inplace=True)

df = pd.concat([pd.get_dummies(df['flag'], prefix='flag'), df], axis=1)
df.drop('flag', axis=1, inplace=True)

df['label'] = np.where(df['label'] == 'smurf', 1, 0.0001)


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(data=scaler.transform(df), columns=df.columns)
df['label'] = np.where(df['label'] == 0, 0.0001, 1)


/home/pollymorphism/miniconda3/lib/python3.7/site-packages/scipy/__init__.py:140: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.3)
  UserWarning)


In [13]:
df['label']

0       0.0001
1       0.0001
2       0.0001
3       0.0001
4       0.0001
5       0.0001
6       0.0001
7       0.0001
8       0.0001
9       0.0001
10      0.0001
11      0.0001
12      0.0001
13      0.0001
14      0.0001
15      0.0001
16      0.0001
17      0.0001
18      0.0001
19      0.0001
20      0.0001
21      0.0001
22      0.0001
23      0.0001
24      0.0001
25      0.0001
26      0.0001
27      0.0001
28      0.0001
29      0.0001
         ...  
9970    1.0000
9971    1.0000
9972    1.0000
9973    1.0000
9974    1.0000
9975    1.0000
9976    1.0000
9977    1.0000
9978    1.0000
9979    1.0000
9980    1.0000
9981    1.0000
9982    1.0000
9983    1.0000
9984    1.0000
9985    1.0000
9986    1.0000
9987    1.0000
9988    1.0000
9989    1.0000
9990    1.0000
9991    1.0000
9992    1.0000
9993    1.0000
9994    1.0000
9995    1.0000
9996    1.0000
9997    1.0000
9998    1.0000
9999    1.0000
Name: label, Length: 10000, dtype: float64

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.25, stratify=df['label'])

In [15]:
print(train['label'].sum())
print(test['label'].sum())

3750.375
1250.125


In [16]:
# In[27]:


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(42)

    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.random.randn(n_h, 1)*0.01
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.random.randn(n_y, 1)*0.01
    
    return W1, b1, W2, b2  

def train_NN(x, y_r, DELTA, b1, b2, w1, w2, rand_state=42):
    np.random.seed(42)
    for i in range(1000):
        y2 = sigmoid(w1.dot(x) + b1)
        y_m = float(sigmoid(w2.dot(y2) + b2))
        delta = float( np.absolute((y_r - y_m)/y_r))
        
        if delta <= DELTA:
            return w1, w2, b1, b2, delta, y_m
        else:
            q3 = float( y_m * (1 - y_m) * (y_r - y_m) )
            d2 = q3 * y2.reshape(1,-1)
            w2 = w2 + d2

          
            q2 = y2 * (1 - y2) * (q3 * w2.reshape(-1, 1))
            d = q2 * x.reshape(1, -1)
            w1 = w1 + d
    return w1, w2, b1, b2, delta, y_m
    
    
    return output
def train_NN_online(X, Y, DELTA = 0.1,
                    hidden_l_size = 3, 
                    show_epochs_stat=False
                   ):
    all_time = time.time()
    W1, b1, W2, b2 = initialize_parameters(X.shape[1], hidden_l_size, 1) 

    for epoch in range(1, 101):
        deltas, y_ms = [], []
        for x, y in zip(X, Y):
            W1, W2, b1, b2, delta, y_m = train_NN(x.reshape(-1, 1), 
                                                  y, 
                                                  DELTA, 
                                                  b1=b1, b2=b2, 
                                                  w1=W1, w2=W2
                                                 )
            deltas.append(delta)
            y_ms.append(y_m)
        if ((epoch == 1) or (epoch % 50 == 0)) and show_epochs_stat:
            print(f'\n-----------------------Epoch {epoch}---------------------------')
            print(pd.DataFrame({
                'Y_r':Y, 
                'deltas':  deltas,
                'Y_ms':y_ms      
            }))
            print('----------------------------------------------------------')   
    print(f'counted for {time.time() - all_time}')
    return W1, W2, b1, b2
    

def predict(x, w1, w2, b1, b2):
    y1 = sigmoid(w1.dot(x) + b1)
    
    output = sigmoid(w2.dot(y1) + b2)
    return float(output)
def predict_online(X, w1, w2, b1, b2):
    y_pred = []
    for x in X:
        y_pred.append( predict(x.reshape(-1, 1), w1, w2, b1, b2) )

    return y_pred




In [17]:
np.random.seed(42)

w1, w2, b1, b2 = train_NN_online(train.iloc[:,:-1].values, train.iloc[:,-1].values, hidden_l_size = 7, show_epochs_stat=True)




-----------------------Epoch 1---------------------------
         Y_r      deltas      Y_ms
0     1.0000    0.097087  0.902913
1     1.0000    0.097087  0.902913
2     0.0001  103.926661  0.010493
3     1.0000    0.097944  0.902056
4     0.0001  104.111727  0.010511
5     1.0000    0.098327  0.901673
6     1.0000    0.098327  0.901673
7     1.0000    0.098327  0.901673
8     0.0001  105.130236  0.010613
9     0.0001   75.024641  0.007602
10    0.0001   60.777704  0.006178
11    0.0001   53.201138  0.005420
12    1.0000    0.098434  0.901566
13    0.0001   89.679209  0.009068
14    1.0000    0.099596  0.900404
15    1.0000    0.099596  0.900404
16    0.0001   75.968842  0.007697
17    1.0000    0.099888  0.900112
18    0.0001   62.202172  0.006320
19    1.0000    0.098261  0.901739
20    0.0001   53.587106  0.005459
21    0.0001   38.758587  0.003976
22    0.0001   43.591966  0.004459
23    1.0000    0.099224  0.900776
24    1.0000    0.099224  0.900776
25    1.0000    0.099224  0.900

In [18]:
pred = predict_online(test.iloc[:,:-1].values, w1, w2, b1, b2)
res = pd.DataFrame(data={
    'real': test.iloc[:,-1].values,
    'pred': pred
})
res[res['pred'] > 1e-2]

,real,pred
0,1.0,0.997325
1,1.0,0.997454
2,1.0,0.997454
6,1.0,0.997454
9,1.0,0.997454
10,1.0,0.997454
13,1.0,0.997454
15,1.0,0.997454
16,1.0,0.997454
17,1.0,0.997454


In [19]:
from sklearn.metrics import confusion_matrix

res['pred'] = np.where(res['pred'] > 1e-2, 1, 0)
res['real'] = np.where(res['pred'] > 1e-2, 1, 0)
a = confusion_matrix(res['real'], res['pred'])
out = f"""
1  {a[0][0]: >3} {a[0][1]: >3}
0  {a[1][0]: >3} {a[1][1]: >3}

     1   0
"""
print(out)


1  1250   0
0    0 1250

     1   0

